In [10]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
import random
from random import randint

We have two types of movies, type 1 and type 2. Each movie has a vector assciotate with them, [x,y]. x and y are binary variables with possible values of 0 and 1. x = 1 means that the film is of type 1, and y = 1 means that the film is of type2.
Each user also has a hidden vector associated with him or her, [b,c], where b and c are preferences for type 1 and type 2 films respectively. $0 \leq b,c \leq 1$ and $b + c = 1$. If a user's perference for a type of film is [a,b], then this user has $a$ chance of accepting recommendations of films of type 1 and $b$ chance of accepting recommendations of films of type 2. This user will rate a film of type a: $a*5 $

In [71]:
# Generate number_of_users of users. Return a number_of_users * 2 array, 
# where each row of the array indicates the user's movie preference.
def generate_Users(number_of_users):
    result = []
    for i in range(0,number_of_users):
        a = random.uniform(0,1)
        result.append([a, 1 - a])
    return(np.array(result))

In [193]:
# Generate a matrix films, where each row of the matrix
# represents the properties of a single film.
def generate_Films(number_of_films):
    result = []
    for i in range(0,number_of_films):
        a = randint(0,1)
        result.append([a, 1-a])
    return(np.array(result))

In [194]:
# Total number of films and user preference matrix
# Percentage is the chance a user has viewed a given film
# Return Value: Each row represents a single film
def user_movie_ratings(films, users, percentage):
    num_films = films.shape[0]
    result = []
    movies = films.tolist()
    Users = users.tolist()
    for film in movies:
        rating = []
        for user in Users:
            ran = random.uniform(0,1)
            if (ran <= percentage):
                t = round(5 * (user[0] * film[0]+ user[1] * film[1])/2)
                if t == 0:
                    t = round(5 * random.uniform(0.1,1))
                rating.append(t)
            else:
                rating.append(0)
        result.append(rating)
    return(np.array(result))

In [195]:
users = generate_Users(6040)
films = generate_Films(3883)
ratings = user_movie_ratings(films, users, 0.05)


In [206]:
def rec(rates):
    user_ratings_mean = np.mean(rates.T, axis = 1)
    R_demeaned = ratings.T - user_ratings_mean.reshape(-1, 1)
    U, sigma, Vt = svds(R_demeaned, k = 50)
    sigma = np.diag(sigma)
    all_user_predicted_ratings = np.round(np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1))
    return(all_user_predicted_ratings)

Question: Should we also recommend videos that users have already viewed? At least for me, Youtube recommends videos that I have already watched all the time.

In [197]:
def recommend_movies(predictions_df, userID,num_recommendations=5):
    temp = predictions_df[userID,]
    recommendations = np.argsort(temp)[-num_recommendations:]
    return recommendations

Initial Recommendations

In [209]:
Recommended = rec(ratings)

In [210]:
initial = []
for i in range(0,users.shape[0]):
    temp = recommend_movies(Recommended, i).tolist()
    type1 = 0
    type2 = 0
    for item in temp:
        if (films[item,0] == 1):
            type1 += 1
        else:
            type2 += 1
    initial.append([type1,type2])

In [242]:
# Simulating what happens when receiving a recommendation
# Take in a list of recommendations
def accepting_recommendations(recommend,initial):
    isChange = False
    for i in range(0,users.shape[0]):
        videos = recommend_movies(recommend,i).tolist()
        type1 = 0
        type2 = 0
        for item in videos:
            if (films[item,0] == 1):
                type1 += 1
            else:
                type2 += 1

        user = users[i,]
        if type1 != initial[i][0] or type2 != initial[i][1]:
            isChange = True
            initial[i][0] = type1
            initial[i][1] = type2
        for video in videos:
            if films[video][0] == 1:
                ran = random.uniform(0,1)
                if ran <= user[0]:
                    mean_rating = np.mean(ratings[:,video])
                    rate = min(round(mean_rating + user[0]),5)
                    ratings[video,i] = rate
            else: 
                ran = random.uniform(0,1)
                if ran <= user[1]:
                    mean_rating = np.mean(ratings[:,video])
                    rate = min(round(mean_rating + user[1]),5)
                    ratings[video,i] = rate
    return(isChange,initial)

In [243]:
accepting_recommendations(Recommended, initial)

(False,
 [[0, 5],
  [2, 3],
  [3, 2],
  [3, 2],
  [3, 2],
  [5, 0],
  [1, 4],
  [2, 3],
  [3, 2],
  [3, 2],
  [3, 2],
  [2, 3],
  [3, 2],
  [3, 2],
  [3, 2],
  [3, 2],
  [3, 2],
  [2, 3],
  [3, 2],
  [3, 2],
  [3, 2],
  [3, 2],
  [1, 4],
  [3, 2],
  [3, 2],
  [3, 2],
  [2, 3],
  [3, 2],
  [3, 2],
  [3, 2],
  [3, 2],
  [3, 2],
  [1, 4],
  [1, 4],
  [2, 3],
  [3, 2],
  [3, 2],
  [1, 4],
  [3, 2],
  [2, 3],
  [2, 3],
  [3, 2],
  [0, 5],
  [3, 2],
  [3, 2],
  [3, 2],
  [3, 2],
  [3, 2],
  [3, 2],
  [3, 2],
  [3, 2],
  [3, 2],
  [3, 2],
  [3, 2],
  [0, 5],
  [0, 5],
  [3, 2],
  [3, 2],
  [0, 5],
  [3, 2],
  [3, 2],
  [3, 2],
  [4, 1],
  [3, 2],
  [3, 2],
  [3, 2],
  [3, 2],
  [0, 5],
  [3, 2],
  [4, 1],
  [3, 2],
  [4, 1],
  [3, 2],
  [3, 2],
  [4, 1],
  [3, 2],
  [3, 2],
  [0, 5],
  [3, 2],
  [2, 3],
  [3, 2],
  [0, 5],
  [3, 2],
  [3, 2],
  [3, 2],
  [3, 2],
  [1, 4],
  [0, 5],
  [3, 2],
  [3, 2],
  [2, 3],
  [2, 3],
  [3, 2],
  [3, 2],
  [1, 4],
  [3, 2],
  [3, 2],
  [3, 2],
  [3, 2],
  

Run Simulation

In [ ]:
i = 0
final_result = []
while True:
    Recommended = rec(ratings)
    Changed, initial = accepting_recommendations(Recommended, initial)
    if not Changed and i != 0:
        final_result = initial
        break
    i = i + 1
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
